In [1]:
# https://medium.com/towards-data-science/transfer-learning-using-keras-d804b2e04ef8

In [1]:
import os
import pandas as pd
import glob 
import random
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

Using TensorFlow backend.


# Preparation

## Inputs

In [2]:
# Fait

# for filename in glob.glob('test/*.jpg'):
#     img = Image.open(filename)
#     img = img.resize((256, 256), Image.ANTIALIAS)
#     img.save(os.path.join("test/resized/", os.path.basename(filename)))

# for filename in glob.glob('train/*.jpg'):
#     img = Image.open(filename)
#     img = img.resize((256, 256), Image.ANTIALIAS)
#     img.save(os.path.join("train/resized/", os.path.basename(filename)))

## Outputs

In [3]:
df = pd.read_csv("labels.csv")
df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10222 entries, 0 to 10221
Data columns (total 2 columns):
id       10222 non-null object
breed    10222 non-null object
dtypes: object(2)
memory usage: 159.8+ KB


## Prepare batches

In [5]:
# for filename in glob.glob('train/resized/*.jpg'):
#     name_img = os.path.basename(filename)[:-4]
#     classe = df[df["id"] == name_img]["breed"].values[0]
#     if not os.path.isdir("train/resized/" + classe):
#         os.mkdir("train/resized/" + classe)
#     os.rename(filename, os.path.join("train", "resized", classe,name_img+".jpg"))

In [6]:
# for folder in os.listdir("train/resized/"):
#     #print(folder)
#     for i in range(10):
#         img = random.choice(os.listdir(os.path.join("train/resized/", folder)))
#         img_path = os.path.join("train/resized/", folder, img)
#         to_path = os.path.join("eval", folder, img)
#         if not os.path.isdir("eval/" + folder):
#             os.mkdir("eval/" + folder)
#         os.rename(img_path, to_path)

# Model

## Training

In [7]:
img_width, img_height = 256, 256
train_data_dir = "train/resized"
validation_data_dir = "eval/"
nb_train_samples = 9022
nb_validation_samples = 1200 
batch_size = 32
epochs = 50

In [8]:
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [9]:
# for layer in model.layers[:5]:   # a tester
#     layer.trainable = False

In [10]:
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(120, activation="softmax")(x)

In [11]:
model_final = Model(input = model.input, output = predictions)

C:\Anaconda501\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [12]:
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [ ]:
latest_save = "vgg19.16.h5"

if os.path.exists(latest_save):
    model_final.load_weights(latest_save)
    print("Weight Loaded...")
else:
    print("No save found...")

In [13]:
checkpoint = ModelCheckpoint("vgg19_1.h5", monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=1, mode='auto')

#### V1

In [117]:
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

In [118]:
train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (img_height, img_width),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (img_height, img_width),
class_mode = "categorical")

Found 9022 images belonging to 120 classes.
Found 1200 images belonging to 120 classes.


In [ ]:
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early])

C:\Anaconda501\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
C:\Anaconda501\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=50, validation_data=<keras.pre..., callbacks=[<keras.ca..., steps_per_epoch=281, validation_steps=1200)`
  import sys


Epoch 1/50
281/281 [==============================] - 585s 2s/step - loss: 4.8387 - acc: 0.0078 - val_loss: 4.7970 - val_acc: 0.0089
Epoch 2/50
281/281 [==============================] - 578s 2s/step - loss: 4.7967 - acc: 0.0105 - val_loss: 4.7931 - val_acc: 0.0101
Epoch 3/50
281/281 [==============================] - 570s 2s/step - loss: 4.7896 - acc: 0.0097 - val_loss: 4.7905 - val_acc: 0.0095
Epoch 4/50
281/281 [==============================] - 569s 2s/step - loss: 4.7867 - acc: 0.0101 - val_loss: 4.7899 - val_acc: 0.0089
Epoch 5/50
281/281 [==============================] - 571s 2s/step - loss: 4.7843 - acc: 0.0110 - val_loss: 4.7896 - val_acc: 0.0085
Epoch 6/50
281/281 [==============================] - 566s 2s/step - loss: 4.7833 - acc: 0.0106 - val_loss: 4.7900 - val_acc: 0.0097
Epoch 7/50
281/281 [==============================] - 571s 2s/step - loss: 4.7820 - acc: 0.0103 - val_loss: 4.7890 - val_acc: 0.0085
Epoch 8/50
281/281 [==============================] - 568s 2s/step - 

#### v2

In [ ]:
# datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True)

In [ ]:
# # (std, mean, and principal components if ZCA whitening is applied)
# datagen.fit(x_train)

# # fits the model on batches with real-time data augmentation:
# model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
#                     steps_per_epoch=len(x_train) / 32, epochs=epochs)

In [ ]:
# for e in range(epochs):
#     print('Epoch', e)
#     batches = 0
#     for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=32):
#         model.fit(x_batch, y_batch)
#         batches += 1
#         if batches >= len(x_train) / 32:
#             # we need to break the loop by hand because
#             # the generator loops indefinitely
#             break

## Reuse Model for prediction

In [2]:
img_width, img_height = 256, 256
test_data_dir = "test/resized"
nb_test_samples = 10357
batch_size = 1
latest_save = "vgg19.16.h5"
steps = nb_test_samples//batch_size+1

model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
# x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(120, activation="softmax")(x)

model_final = Model(input = model.input, output = predictions)

model_final.load_weights(latest_save)



C:\Anaconda501\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [3]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(test_data_dir, target_size = (img_height, img_width), class_mode = None)

Found 0 images belonging to 0 classes.


In [4]:
test_generator.class_indices

{}

In [ ]:
y_pred = model_final.predict_generator(test_generator, steps=1)
print(y_pred)

Found 0 images belonging to 0 classes.
